In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [12]:
# Training data augmentation
transform_train=transforms.Compose([
    transforms.RandomCrop(32,padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.2,0.2,0.2)),
])

In [7]:
# Normalize the test set same as training set without augmentation
transform_test=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.2,0.2,0.2))
])

In [15]:
# Load the training dataset
trainset=torchvision.datasets.CIFAR10(
        root='./data',train=True,download=True,transform=transform_train)

trainloader=torch.utils.data.DataLoader(trainset,batch_size=4,shuffle=True,num_workers=4)


Files already downloaded and verified


In [21]:
# Load the testing dataset
testset=torchvision.datasets.CIFAR10(
    root='./data',train=False,download=True,transform=transform_test)
testloader=torch.utils.data.DataLoader(testset,batch_size=4,shuffle=False,num_workers=4)

Files already downloaded and verified


In [37]:
# Define a Convulution Neural Network

import torch
import torch.nn as nn

class CNN(nn.Module):
    """Convultion Neural Network"""
    
    def __init__(self):
        """CNN Builder"""
        super(CNN,self).__init__()
        self.conv_layer=nn.Sequential(
            # Conv Layer 1
            nn.Conv2d(in_channels=3,out_channels=32,kernel_size=3,padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=3,out_channels=64,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
            
            # Conv Layer 2
            nn.Conv2d(in_channels=64,out_channels=128,kernel_size=3,padding=1),
                     nn.BatchNorm2d(128),
                     nn.ReLU(inplace=True),
                     nn.Conv2d(in_channels=128,out_channels=128,kernel_size=3,padding=1),
                     nn.ReLU(inplace=True),
                     nn.MaxPool2d(kernel_size=2,stride=2),
                     nn.Dropout(p=0.1),
            
            # Conv Layer 3
            nn.Conv2d(in_channels=128,out_channels=256,kernel_size=3,padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=256,out_channels=256,kernel_size=3,padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2,stride=2),
        )
        self.fc_layer=nn.Sequential(
            nn.Dropout(p=0.1),
            nn.Linear(4096,1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024,512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.1),
            nn.Linear(512,10))
        
        def forward(self,x):
            """Perform forward"""
            # Conv layer
            x=self.conv_layer(x)
            
            # Flatten 
            x=x.view(x.size(0),-1)
            
            # fc layer
            x=self.fc_layer(x)
            
            return x

net=CNN()

In [40]:
# Use GPU for training
device='cuda:0'
net.to(device)

CNN(
  (conv_layer): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Dropout(p=0.1, inplace=False)
    (13): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

In [38]:
# Define a Loss Function and Optimizer
# I choose to use a Classification Cross-Entropy loss and Adam optimizer

import torch.optim as optim

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(net.parameters(),lr=0.001)

In [45]:
for epoch in range(10):
    running_loss=0.
    batch_size=100
    
    for i, data in enumerate(trainloader,0):
        inputs,labels=data
        inputs,labels=inputs.to(device),labels.to(device)
        
        optimizer.zero_grad()
        
        outputs=net(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()
        
        print('[%d,%5d] loss:%.4f'%(epoch+1,(i+1)*batch_size,loss.item()))

print('Training Finished!')

NotImplementedError: 